## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [86]:
from collections import Counter
import itertools

counts = Counter(list(itertools.chain.from_iterable(sentences_ted)))

ted_top1000 = counts.most_common()[:1000]
counts_ted_top1000 = list(map(lambda x: x[1], ted_top1000))
words_top_ted = list(map(lambda x: x[0], ted_top1000))

In [87]:
ted_top1000

[('the', 207748),
 ('and', 149305),
 ('to', 125169),
 ('of', 114818),
 ('a', 105399),
 ('that', 95146),
 ('i', 83180),
 ('in', 78070),
 ('it', 74738),
 ('you', 70923),
 ('we', 67629),
 ('is', 63251),
 ('s', 57156),
 ('this', 49241),
 ('so', 37014),
 ('they', 33102),
 ('was', 30806),
 ('for', 29713),
 ('are', 27995),
 ('have', 27344),
 ('but', 26732),
 ('what', 26519),
 ('on', 25962),
 ('with', 24706),
 ('can', 23377),
 ('t', 22757),
 ('about', 21246),
 ('there', 21041),
 ('be', 20201),
 ('as', 19488),
 ('at', 19216),
 ('all', 19021),
 ('not', 18626),
 ('do', 17928),
 ('my', 17908),
 ('one', 17551),
 ('re', 17012),
 ('people', 16723),
 ('like', 16273),
 ('if', 15868),
 ('from', 15452),
 ('now', 14387),
 ('our', 14061),
 ('he', 13986),
 ('an', 13917),
 ('just', 13896),
 ('these', 13882),
 ('or', 13864),
 ('when', 13278),
 ('because', 12879),
 ('very', 12363),
 ('me', 12302),
 ('out', 12163),
 ('by', 11935),
 ('them', 11595),
 ('how', 11576),
 ('know', 11506),
 ('up', 11445),
 ('going', 1

Plot distribution of top-1000 words

In [46]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [48]:
from gensim.models import Word2Vec

In [79]:
model_ted = Word2Vec(sentences_ted, min_count=10)
len(model_ted.wv.vocab)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [82]:
model_ted.most_similar("man")

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.8464910984039307),
 ('guy', 0.8140854239463806),
 ('lady', 0.7679780125617981),
 ('girl', 0.7450454831123352),
 ('gentleman', 0.7368344068527222),
 ('boy', 0.7304062247276306),
 ('soldier', 0.7235003709793091),
 ('surgeon', 0.6928210854530334),
 ('kid', 0.6904268264770508),
 ('poet', 0.6590760350227356)]

In [83]:
model_ted.most_similar("computer")

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('machine', 0.7886710166931152),
 ('software', 0.7199345231056213),
 ('robot', 0.7167546153068542),
 ('device', 0.6893418431282043),
 ('3d', 0.6661658883094788),
 ('program', 0.6563683748245239),
 ('video', 0.6511272192001343),
 ('camera', 0.6501621007919312),
 ('interface', 0.6223925352096558),
 ('chip', 0.6209701299667358)]

In [84]:
model_ted.most_similar('lunch')

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('dinner', 0.8022578358650208),
 ('breakfast', 0.7379685640335083),
 ('home', 0.6567416191101074),
 ('milk', 0.6517292261123657),
 ('bed', 0.6445494294166565),
 ('hospital', 0.6325748562812805),
 ('wedding', 0.6278756856918335),
 ('mom', 0.6245189905166626),
 ('desk', 0.6244857907295227),
 ('clinic', 0.6197742819786072)]

In [85]:
model_ted.most_similar('robot')

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('device', 0.8231595754623413),
 ('machine', 0.7666153907775879),
 ('camera', 0.7250755429267883),
 ('mouse', 0.720550000667572),
 ('computer', 0.7167547345161438),
 ('sensor', 0.6915122866630554),
 ('trick', 0.6891667246818542),
 ('button', 0.6685299277305603),
 ('keyboard', 0.6634910106658936),
 ('game', 0.6483930945396423)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [88]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [89]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [90]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [102]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [103]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [104]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [105]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [108]:
sentences_wiki_tokens = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki_tokens.append(tokens)

In [119]:
counts = Counter(list(itertools.chain.from_iterable(sentences_wiki_tokens)))

wiki_top1000 = counts.most_common()[:1000]
counts_wiki_top1000 = list(map(lambda x: x[1], wiki_top1000))
words_top_wiki = list(map(lambda x: x[0], wiki_top1000))

In [112]:
wiki_top1000

[('the', 1286941),
 ('of', 547118),
 ('and', 500509),
 ('in', 434464),
 ('to', 398938),
 ('a', 358713),
 ('was', 215289),
 ('s', 164717),
 ('on', 153988),
 ('as', 145440),
 ('for', 139884),
 ('that', 138257),
 ('with', 130088),
 ('by', 126016),
 ('is', 103078),
 ('at', 92976),
 ('his', 92889),
 ('he', 91738),
 ('from', 89151),
 ('it', 88773),
 ('were', 70938),
 ('an', 64424),
 ('had', 58941),
 ('which', 55120),
 ('be', 47262),
 ('this', 43218),
 ('but', 42392),
 ('are', 40694),
 ('first', 40364),
 ('not', 39538),
 ('their', 38940),
 ('after', 38242),
 ('one', 37612),
 ('her', 37097),
 ('also', 36986),
 ('its', 36726),
 ('they', 34885),
 ('two', 34370),
 ('or', 33696),
 ('have', 32948),
 ('who', 31762),
 ('new', 30876),
 ('been', 30731),
 ('has', 30367),
 ('she', 29458),
 ('when', 27131),
 ('time', 26398),
 ('during', 26266),
 ('other', 25644),
 ('would', 25338),
 ('into', 24926),
 ('all', 24299),
 ('i', 23586),
 ('more', 22979),
 ('while', 21430),
 ('over', 21059),
 ('him', 19894),
 ('

In [113]:
model_wiki = Word2Vec(sentences_wiki_tokens, min_count=10)
len(model_wiki.wv.vocab)

48535

In [114]:
model_ted.most_similar("man")

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.8464910984039307),
 ('guy', 0.8140854239463806),
 ('lady', 0.7679780125617981),
 ('girl', 0.7450454831123352),
 ('gentleman', 0.7368344068527222),
 ('boy', 0.7304062247276306),
 ('soldier', 0.7235003709793091),
 ('surgeon', 0.6928210854530334),
 ('kid', 0.6904268264770508),
 ('poet', 0.6590760350227356)]

In [115]:
model_ted.most_similar("computer")

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('machine', 0.7886710166931152),
 ('software', 0.7199345231056213),
 ('robot', 0.7167546153068542),
 ('device', 0.6893418431282043),
 ('3d', 0.6661658883094788),
 ('program', 0.6563683748245239),
 ('video', 0.6511272192001343),
 ('camera', 0.6501621007919312),
 ('interface', 0.6223925352096558),
 ('chip', 0.6209701299667358)]

In [116]:
model_ted.most_similar("human")

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sentient', 0.6170675158500671),
 ('biological', 0.5248968005180359),
 ('conscious', 0.5068625807762146),
 ('evolution', 0.5068567395210266),
 ('humans', 0.5054645538330078),
 ('individual', 0.5024294853210449),
 ('mental', 0.494734525680542),
 ('function', 0.49288511276245117),
 ('physical', 0.4906421899795532),
 ('compassion', 0.482526957988739)]

In [117]:
model_ted.most_similar("robot")

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('device', 0.8231595754623413),
 ('machine', 0.7666153907775879),
 ('camera', 0.7250755429267883),
 ('mouse', 0.720550000667572),
 ('computer', 0.7167547345161438),
 ('sensor', 0.6915122866630554),
 ('trick', 0.6891667246818542),
 ('button', 0.6685299277305603),
 ('keyboard', 0.6634910106658936),
 ('game', 0.6483930945396423)]

#### t-SNE visualization

In [120]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

/Users/miguel/anaconda2/envs/oxford-deep-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [121]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)